<a href="https://colab.research.google.com/github/jiayeelim/AIROST_BrainTumorClassification/blob/main/2_Pickle_Data_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from PIL import Image
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
import random
import tensorflow as tf

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


For Training Data

In [ ]:
TEST_DIR = '/content/gdrive/My Drive/AIROST/Dataset/Testing' # test data folder
TRAIN_DIR = '/content/gdrive/My Drive/AIROST/Dataset/Training' # train data folder
IMG_SIZE = 150 # image size
CATEGORIES = ["Train-glioma-output","Train-meningioma-output","Train-no_tumor-output","Train-pituitary-output"] #"Train-glioma-output",

In [ ]:
training_data = []

def create_training_data():

    for category in CATEGORIES:

        path = os.path.join(TRAIN_DIR,category)  # create path
        class_num = CATEGORIES.index(category)  # get the classification

        for img in tqdm(os.listdir(path)):
          # iterate over each image
          img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_COLOR)  # convert to array
          new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize to normalize data size
          training_data.append([new_array, class_num])  # add this to our training_data

    random.shuffle(training_data)
    #np.save('train_data1.npy', training_data)

create_training_data()
np.save('/content/gdrive/My Drive/AIROST/Dataset/train_data.npy', training_data)
print(len(training_data))

print("train")
print()
X_train = np.array([i[0] for i in training_data]).reshape(-1,IMG_SIZE,IMG_SIZE,3)
Y_train = [i[1] for i in training_data]

import pickle

pickle_out = open("/content/gdrive/My Drive/AIROST/Dataset/X_train.pickle","wb")
pickle.dump(X_train, pickle_out)
pickle_out.close()

pickle_out = open("/content/gdrive/My Drive/AIROST/Dataset/Y_train.pickle","wb")
pickle.dump(Y_train, pickle_out)
pickle_out.close()

100%|██████████| 17348/17348 [01:02<00:00, 275.73it/s]
/usr/local/lib/python3.10/dist-packages/numpy/lib/npyio.py:521: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


71791
train



File too big. Try to use batch

In [ ]:
# Use batch to save the file
import numpy as np
import pickle
from tqdm import tqdm
import cv2
import os

def data_generator(directory, categories, img_size, batch_size):
    for category in categories:
        path = os.path.join(directory, category)
        class_num = categories.index(category)

        batch_data = []
        for img in tqdm(os.listdir(path)):
            img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_COLOR)
            new_array = cv2.resize(img_array, (img_size, img_size))
            batch_data.append((new_array, class_num))

            if len(batch_data) == batch_size:
                # Yield the current batch and reset
                yield batch_data
                batch_data = []

        # Yield the remaining data in the last batch
        if batch_data:
            yield batch_data

def save_batches_to_pickle(directory, categories, img_size, batch_size, pickle_path):
    data_gen = data_generator(directory, categories, img_size, batch_size)

    for idx, batch_data in enumerate(data_gen):
        X_batch = np.array([data[0] for data in batch_data]).reshape(-1, img_size, img_size, 3)
        pickle_out = open(f"{pickle_path}_batch{idx}.pickle", "wb")
        pickle.dump(X_batch, pickle_out)
        pickle_out.close()


# Assuming the values of TRAIN_DIR, IMG_SIZE, BATCH_SIZE, and PICKLE_PATH are already defined
TRAIN_DIR = '/content/gdrive/My Drive/AIROST/Dataset/Training' # train data folder
IMG_SIZE = 150 # image size
BATCH_SIZE = 30000
categories = ["Train-glioma-output", "Train-meningioma-output", "Train-no_tumor-output", "Train-pituitary-output"]

save_batches_to_pickle(TRAIN_DIR, categories, IMG_SIZE, BATCH_SIZE, "/content/gdrive/My Drive/AIROST/Dataset/X_train")


100%|██████████| 17348/17348 [08:53<00:00, 32.52it/s] 


In [ ]:
#Combine the file
import numpy as np
import pickle
import os

def combine_batches(pickle_path, num_batches):
    combined_data = []

    for i in range(num_batches):
        batch_file = f"{pickle_path}_batch{i}.pickle"

        with open(batch_file, 'rb') as file:
            batch_data = pickle.load(file)
            combined_data.extend(batch_data)

    return combined_data

# Example usage
pickle_path = "/content/gdrive/My Drive/AIROST/Dataset/X_train"
num_batches = 4  # Change this to the actual number of batches

all_data = combine_batches(pickle_path, num_batches)

#IMG_SIZE = 150 # image size

# Convert the list of tuples to a numpy array
#X_train_combined = np.array([data[0] for data in all_data]).reshape(-1, IMG_SIZE, IMG_SIZE, 3)

#print(X_train_combined.shape)
#print(X_train_combined.size)

# Save the combined data to a single pickle file
combined_pickle_path = "/content/gdrive/My Drive/AIROST/Dataset/X_train_combined.pickle"
with open(combined_pickle_path, 'wb') as file:
    pickle.dump(all_data, file)


Retry Using Original Data only<br>
Exclude Augmented Data


In [ ]:
import os
from PIL import Image
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
import random
import tensorflow as tf

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
TEST_DIR = '/content/gdrive/My Drive/AIROST/Dataset/Testing' # test data folder
TRAIN_DIR = '/content/gdrive/My Drive/AIROST/Dataset/Training' # train data folder
IMG_SIZE = 150 # image size
CATEGORIES = ["glioma_tumor","meningioma_tumor","no_tumor","pituitary_tumor"] #"Train-glioma-output",

In [ ]:
training_data = []

def create_training_data():

    for category in CATEGORIES:

        path = os.path.join(TRAIN_DIR,category)  # create path
        class_num = CATEGORIES.index(category)  # get the classification

        for img in tqdm(os.listdir(path)):
          # iterate over each image
          img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_COLOR)  # convert to array
          new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize to normalize data size
          training_data.append([new_array, class_num])  # add this to our training_data

    random.shuffle(training_data)
    #np.save('train_data1.npy', training_data)

create_training_data()
np.save('/content/gdrive/My Drive/AIROST/Dataset/train_data_new.npy', training_data)
print(len(training_data))

print("train")
print()
X_train = np.array([i[0] for i in training_data]).reshape(-1,IMG_SIZE,IMG_SIZE,3)
Y_train = [i[1] for i in training_data]

import pickle

pickle_out = open("/content/gdrive/My Drive/AIROST/Dataset/X_train_new.pickle","wb")
pickle.dump(X_train, pickle_out)
pickle_out.close()

pickle_out = open("/content/gdrive/My Drive/AIROST/Dataset/Y_train_new.pickle","wb")
pickle.dump(Y_train, pickle_out)
pickle_out.close()

100%|██████████| 827/827 [00:15<00:00, 51.73it/s] 
/usr/local/lib/python3.10/dist-packages/numpy/lib/npyio.py:521: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


2870
train



In [ ]:
X_train.shape

(2870, 150, 150, 3)